<a href="https://colab.research.google.com/github/Chirag7077/Recommendation-System/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chirag7077","key":"0f28c077fb10bc6ed5b557a7306ccf55"}'}

In [9]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d arashnic/book-recommendation-dataset

 38% 9.00M/23.8M [00:00<00:00, 74.3MB/s]
100% 23.8M/23.8M [00:00<00:00, 135MB/s] 


In [11]:
!unzip /content/book-recommendation-dataset.zip

Archive:  /content/book-recommendation-dataset.zip
  inflating: Books.csv               
  inflating: Ratings.csv             
  inflating: Users.csv               


In [12]:
#!mkdir /content/Book_Recommendation_System
!cp /content/Books.csv /content/Book_Recommendation_System/
!cp /content/Ratings.csv /content/Book_Recommendation_System/
!cp /content/Users.csv /content/Book_Recommendation_System/

In [13]:
!rm /content/Books.csv
!rm /content/Users.csv 
!rm /content/Ratings.csv 

In [14]:
# Imports
import pandas as pd
import numpy as np

# Import the datasets
books = pd.read_csv('/content/Book_Recommendation_System/Books.csv')
users = pd.read_csv('/content/Book_Recommendation_System/Users.csv')
ratings = pd.read_csv('/content/Book_Recommendation_System/Ratings.csv')

print('Books: ' + str(books.shape))
print('Users: ' + str(users.shape))
print('Ratings: ' + str(ratings.shape))

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Books: (271360, 8)
Users: (278858, 3)
Ratings: (1149780, 3)


In [15]:
# Check Missing values
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [16]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [17]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [18]:
# Check Duplicated values
books.duplicated().sum()
users.duplicated().sum()
ratings.duplicated().sum()

0

## **Popularity Based Recommendation System**

In [26]:
# Display the top 50 books with highest average ratings. We will consider only those books which have atleast 250 ratings given to them

# Merge Ratings and Books
ratings_books = ratings.merge(books, on = 'ISBN')
ratings_books.shape

(1031136, 10)

In [27]:
num_ratings_df = ratings_books.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_ratings_df.rename(columns = {'Book-Rating' : 'num_ratings'}, inplace=True)
num_ratings_df.head()

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [28]:
avg_ratings_df = ratings_books.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_ratings_df.rename(columns = {'Book-Rating' : 'avg_ratings'}, inplace=True)
avg_ratings_df.head()

,Book-Title,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [29]:
popularity_df = num_ratings_df.merge(avg_ratings_df, on = 'Book-Title')
popularity_df.head()

,Book-Title,num_ratings,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00
2,Apple Magic (The Collector's series),1,0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.00
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.00


In [30]:
popularity_df = popularity_df[popularity_df['num_ratings']>=250].sort_values('avg_ratings', ascending=False).head(50)

In [31]:
popularity_df = popularity_df.merge(books, on = 'Book-Title').drop_duplicates('Book-Title')[['Book-Title', 'Book-Author', 'Image-URL-M', 'num_ratings', 'avg_ratings']]

In [32]:
popularity_df.head()

,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453


## Collaborative Filtering Based Recommendation System

In [38]:
x = ratings_books.groupby('User-ID').count()['Book-Rating'] > 200
users_to_consider = x[x].index

In [42]:
filtered_rating = ratings_books[ratings_books['User-ID'].isin(users_to_consider)]
filtered_rating.shape

(474007, 10)

In [45]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [48]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
final_ratings.shape

(58586, 10)

In [52]:
pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
pt.shape

(706, 810)

In [55]:
pt.fillna(0, inplace=True)
pt.head(2)

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(pt)

In [72]:
def recommendBooks(bookName):
  # fetch the index of bookName
  suggestions = []
  index = np.where(pt.index == bookName)[0][0]
  similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x:x[1], reverse = True)[1:6]
  for i in similar_items:
    suggestions.append(pt.index[i[0]])
  return suggestions

In [74]:
print(recommendBooks('Animal Farm'))

['1984', 'Angus, Thongs and Full-Frontal Snogging: Confessions of Georgia Nicolson', 'Midnight', 'Second Nature', 'Call of the Wild']
